### ChallengeB: move the camera such as the ROI is in the center of the frame

#### Note: For calculations and notations please refer to the paper attached.

In [1]:
import numpy as np

In [2]:
class Camera():
    def __init__(self):
        #initializa pan and tilt angles of the class Camera to 0 rad.
        self.sensorSize = 1 / 2.8 # [inch]
        self.opticalZoom = 25   # 25X
        self.focalLengthMin = 4.8 # [mm]
        self.focalLengthMax = 120 
        self.resolution = (1920, 1080)
        self.pixelWidth  = 2.9e-3    # [mm] - assumption: pixel size = 2.9um x 2.9 um
        self.pixelHeight  = 2.9e-3  # [mm] - assumption: pixel size = 2.9um x 2.9 um
        
        self.alpha = 0.0 #initialize pan 
        self.beta = 0.0  #initialize tilt
    
    # Utility Func.
    def calcFocalLength(self, zoom):
        """
        Compute focal length given zoom level in %       
        Args:
            zoom: current zoom level[%]
        Returns:
            FocalLength : corresponding zoom [mm] 
        """
        # changing the zoom from % to X
        opticalZoomSpan = self.opticalZoom - 1
        zoom_X = opticalZoomSpan * zoom / 100 + 1
        
        # calculating the focal length in mm
        focalLengthSpan = self.focalLengthMax - self.focalLengthMin 
        m = focalLengthSpan / opticalZoomSpan  #slope of the line
        b = self.focalLengthMin - m            #line bias 
        FocalLength = m * zoom_X + b

        return FocalLength


    def pixel_to_steps (self, roi_pixel_coord, currentZoom):
        """
        Coverts pixel coordinates to desired pan and tilt angle 
        that would would center the Water Hole (after panning and tilting the camera), 
        given the camera's current pan & tilt
        
        Args:
            roi_pixel_coord: (px, tx) representing pixel coordinate of the ROI
            currentZoom: 0 to 100 representing current zoom level
        
        Returns:
            desired pan (alpha_des) [rad] 
            desired tilt (beta_des) [rad]
        """
        R_y = np.array([[np.cos(self.alpha), 0, np.sin(self.alpha)], [0, 1, 0], [-np.sin(self.alpha), 0, np.cos(self.alpha)]])
        R_x = np.array([[1, 0, 0], [0, np.cos(self.beta), -np.sin(self.beta)], [0, np.sin(self.beta), np.cos(self.beta)]])
        
        M_RotCI = R_y @ R_x    # MRotCI transforms Camera{C} coordinates into Inertia{I} coordinates.
        
        f = self.calcFocalLength(currentZoom)
        x = roi_pixel_coord[0]
        y = roi_pixel_coord[1]
        x_0 =  self.resolution[0] / 2   # image midpoint x pixel coordinate 
        y_0 =  self.resolution[1] / 2   # image midpoint y pixel coordinate
        
        f_x = self.pixelWidth / f       # const.
        f_y = self.pixelHeight / f      #const.
        
        d_C = np.array([f_x * (x - x_0), f_y * (y - y_0), 1])   # Water Hole coordinate in Camera{C} frame.
        
        d_I = M_RotCI @ d_C # Transformation of Water Hole coordination from Camera{C} frame to Inertia{C} frame.       

        alpha_des = -np.arctan2(d_I[0], d_I[2])
        beta_des = np.arctan2(d_I[1], d_I[2] / np.cos(alpha_des))
        
        return (alpha_des, beta_des)

    def move_camera(self, pt_coordinate):
        
        
        
        # the number of movement step 
        pan_step =  round(pt_coordinate[0] * 180 / np.pi)  # rad to deg conversion-each degree = 1 step
        tilt_step = round(pt_coordinate[1] * 180 / np.pi)   # rad to deg conversion-each degree = 1 step 
        
        # the camera position after movement
        self.alpha = pt_coordinate[0]
        self.beta = pt_coordinate[1]
        
        return (pan_step, tilt_step)

In [9]:
camera = Camera()
roi_pixel_coord = (1920, 1080) 
currentZoom = 10
pt_coordinate = camera.pixel_to_steps(roi_pixel_coord, currentZoom)
pt_steps = camera.move_camera(pt_coordinate) 

print(pt_coordinate)
print(pt_steps)

(-0.16896181431958118, 0.09430886119958314)
(-10, 5)
